<a href="https://colab.research.google.com/github/AnoushkaVijay/Leukemia_GAN/blob/main/Featurization_Train_Validation_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Please replace the brackets below with the drive location of your folder which included subfolders for images
# Sample path: /content/drive/My Drive/ImageClassification
PATH = ''

In [ ]:
def get_optimizer(optimizer_name, learning_rate):
    # Import keras optimizers
    from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, Ftrl, Nadam, RMSprop, SGD
    print('Selected Optimizer', optimizer_name)
    switcher = {
        'Adadelta': Adadelta(lr=learning_rate),
        'Adagrad': Adagrad(lr=learning_rate),
        'Adam': Adam(lr=learning_rate),
        'Adamax': Adamax(lr=learning_rate),
        'FTRL': Ftrl(lr=learning_rate),
        'NAdam': Nadam(lr=learning_rate),
        'RMSprop': RMSprop(lr=learning_rate),
        'Gradient Descent': SGD(lr=learning_rate)
    }
    # If optimizer_name is empty, Adam will be return as default optimizer
    return switcher.get(optimizer_name, Adam(lr=learning_rate))


In [ ]:
def convert_tf_dataset(PATH, model):
    # This function passes all images provided in PATH
    # and passes them through the model.
    # The result is a featurized image along with labels
    data = []
    IMG_SIZE = (224, 224)
    file_list = []

    # Get the list of subfolders
    sub_dirs = next(os.walk(PATH))[1]
    print(sub_dirs)
    num_images = 0

    # Create a list of lists
    # Number of lists is same as the number of subfolders
    # Number of items in the sub-list is the number of
    # images in each sub-folder
    for category in sub_dirs:
        files = next(os.walk(PATH + '/' + category), (None, None, []))[2]
        filenames = [PATH + '/' + category + '/' + file for file in files]
        num_images += len(filenames)
        file_list.append(filenames)

    labels = []
    # Every image is pre-processed and passed thought the model
    # Label is created for every image
    for category in file_list:
        for img_path in category:
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_batch = np.expand_dims(img_array, axis=0)
            img_preprocessed = preprocess_input(img_batch)
            data.append(model.predict(img_preprocessed))
            labels.append(img_path.split('/')[-2])


    # Make sure dimensions are (num_samples, 1280)
    data = np.squeeze(np.array(data))
    labels = np.reshape(labels, (-1,1))
    return data, labels

In [ ]:

# Import packages needed to create a image classification model
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from keras.applications.resnet import preprocess_input
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model

from keras.layers import Dense,GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from tensorflow import keras

IMG_SIZE = (224, 224)


# Download the model, valid alpha values [0.25,0.35,0.5,0.75,1]
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
# Add average pooling to the base
x = base_model.output
x = GlobalAveragePooling2D()(x)
model_frozen = Model(inputs=base_model.input,outputs=x)

# Get the transformed features from the dataset
# TODO: This can be moved to the FE stage of the pipeline
# label_map is not used anywhere right now. it has information
# about which label is mapped to which number
data, labels = convert_tf_dataset(PATH, model_frozen)
# Shuffle the dataset for training
shuffler = np.random.permutation(len(data))
data_shuffled = data[shuffler]
labels_shuffled = labels[shuffler]
print(data_shuffled)
num_features = data_shuffled.shape[1]

In [ ]:
num_features

1280

In [ ]:
import pandas as pd
feature_names = []
for a in range(0,num_features):
  feature_names.append('feature_' + str(a))
feature_names.append('label')
df = pd.DataFrame(data=np.hstack((data_shuffled,labels_shuffled)), columns=feature_names)
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1271,feature_1272,feature_1273,feature_1274,feature_1275,feature_1276,feature_1277,feature_1278,feature_1279,label
0,0.010298399,0.45297498,0.0,0.5555206,0.008394484,2.2204435,1.3242031,0.0,0.0,0.788158,...,0.64402604,0.0,0.0,0.0,0.0,0.0,0.48082033,0.13815227,0.3367185,Pitted
1,0.08914236,0.039412417,0.6327165,0.08082508,0.027472915,1.0618368,1.0789471,0.0,0.04429935,0.28672656,...,0.02375786,0.0,0.0,0.7215307,0.0,0.0,0.0,0.0,0.09490932,Scratches
2,0.02005255,0.0,0.07667999,0.4759574,0.0,0.846536,0.028269388,0.20538959,0.61647254,1.1578925,...,0.026453577,0.05088169,0.0,0.023354826,0.0,0.0,0.0,0.0,0.01674572,Rolled
3,0.029670885,0.052766692,0.042521317,1.9113878,0.07329471,1.2102338,0.0,0.0,0.0,0.651231,...,0.0,0.0,0.0,0.033115223,0.0,0.0,0.0,0.005167322,0.056232087,Rolled
4,0.05710411,0.0,0.011939455,2.3239317,0.15070464,2.662899,1.0494467,0.0,0.0,0.45937523,...,0.23032145,0.0,0.0,0.0173117,0.0,0.0,0.105863415,0.047811322,0.8678681,Crazing


In [ ]:
# Save the csv
df.to_csv('', index=False)